In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf

In [0]:

new_model=tf.keras.models.load_model('/content/drive/My Drive/asm.h5')
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 196, 196, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 98, 98, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 614656)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               78676096  
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [0]:
tf.function()

In [0]:
# convert the model to TFLite
!mkdir "tflite_models"
TFLITE_MODEL = "tflite_models/plant_disease_model.tflite"


# Get the concrete function from the Keras model.
run_model = tf.function(lambda x : reloaded(x))

# Save the concrete function.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(new_model.inputs[0].shape, new_model.inputs[0].dtype)
)

# Convert the model to standard TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converted_tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(converted_tflite_model)

In [0]:
tflite_converter = tf.lite.TFLiteConverter.from_keras_model_file('/content/drive/My Drive/asm.h5')
tflite_model = tflite_converter.convert()
open("/content/drive/My Drive/tf_lite_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.


314790652

In [0]:
import tensorflow as tf
interpreter = tf.lite.Interpreter("/content/drive/My Drive/tf_lite_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [0]:
input_details

[{'dtype': numpy.float32,
  'index': 6,
  'name': 'conv2d_input',
  'quantization': (0.0, 0),
  'shape': array([  1, 200, 200,   3], dtype=int32)}]

In [0]:
output_details

[{'dtype': numpy.float32,
  'index': 12,
  'name': 'dense_1/Softmax',
  'quantization': (0.0, 0),
  'shape': array([ 1, 13], dtype=int32)}]